<a href="https://colab.research.google.com/github/gagan3012/zeroshot-learning/blob/master/Zero_Shot_learning_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv("/content/bquxjob_3bc46ddf_17ce26263bb.csv")
df.head()

In [ ]:
!pip install newspaper3k==0.2.8

In [ ]:
from bs4 import BeautifulSoup
import newspaper


def get_article_metadata_newspaper(df):
    """
    Get/extract the metadata of news article, including, article heading and main content, from it's URL.
    Arguments:
        url: (str) link of news article (starting with http:// or https://) 
    Returns:
        dict:
            title: (str) extracted heading of news article
            text: (str) extracted main content of news article 
    """

    df = df.drop_duplicates(subset=['SOURCEURL'],ignore_index = True)
    df["text"] = 0
    df["title"]= 0

    for i in range(len(df)):
      url = df['SOURCEURL'][i]
      article = newspaper.Article(url)

      try:
        article.download()
        article.parse()
      except:
        continue

      df["text"][i] = article.text
      df["title"][i] = article.title
 
    return df
    

In [ ]:
df_new = get_article_metadata_newspaper(df)

In [ ]:
df_new.head()

In [ ]:
!pip install transformers

In [ ]:
len(df_new["text"][0])

In [ ]:
import re
def stringupdate(i):
  strList = df_new["Themes"][i].split(";")

  strList = list(map(lambda x: str(x).replace('WB_', ''), strList))

  strList = list(map(lambda x: re.sub('^\d+', '', x), strList))

  strList = list(map(lambda x: str(x).replace('_', '', 1), strList))

  strList = list(filter(None, strList))

  return strList

##### ZSL
This is the checkpoint for bart-large after being trained on the MultiNLI (MNLI) dataset.

Additional information about this model:

The bart-large model page
BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension
BART fairseq implementation
NLI-based Zero Shot Text Classification
Yin et al. proposed a method for using pre-trained NLI models as a ready-made zero-shot sequence classifiers. The method works by posing the sequence to be classified as the NLI premise and to construct a hypothesis from each candidate label. For example, if we want to evaluate whether a sequence belongs to the class "politics", we could construct a hypothesis of This text is about politics.. The probabilities for entailment and contradiction are then converted to label probabilities.

This method is surprisingly effective in many cases, particularly when used with larger pre-trained models like BART and Roberta. See this blog post for a more expansive introduction to this and other zero shot methods, and see the code snippets below for examples of using this model for zero-shot classification both with Hugging Face's built-in pipeline and with native Transformers/PyTorch code.

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [ ]:
sequence_to_classify =df_new["title"][3]
candidate_labels = stringupdate(3)
classifier(sequence_to_classify, candidate_labels, multi_label=True)